## Introdução do problema
Neste notebook será realizada a classificação do sentimento obtidos através de letras musicais, agrupados por gênero. com os resultados poderemos ver os valores de sentimentos positivos negativos ou neutros para cada gênero.

## Principais atributos a serem trabalhados

genre: Genero da musica </br>
lyrics: Letra completa da música</br>
track_name: nome da música</br>

## Importação das bibliotecas 

In [1]:
import pandas as pd
import os
from google.colab import drive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Conexão com o drive e carregamento da base de dados

Definição do path do drive

In [2]:
drive.mount("/content/drive")
path = '/content/drive/MyDrive/Base de musicas/'

Caminhos de cada base separados

In [5]:
arquivo_base_musicas = os.path.join(path,"music_dataset.csv")

Carregamento das bases de dados

In [6]:
dataset = pd.read_csv(arquivo_base_musicas)

In [7]:
dataset.head()

,Unnamed: 0,artist_name,track_name,release_date,genre,lyrics,len,dating,violence,world/life,...,sadness,feelings,danceability,loudness,acousticness,instrumentalness,valence,energy,topic,age
0,0,mukesh,mohabbat bhi jhoothi,1950,pop,hold time feel break feel untrue convince spea...,95,0.000598,0.063746,0.000598,...,0.380299,0.117175,0.357739,0.454119,0.997992,0.901822,0.339448,0.137110,sadness,1.0
1,4,frankie laine,i believe,1950,pop,believe drop rain fall grow believe darkest ni...,51,0.035537,0.096777,0.443435,...,0.001284,0.001284,0.331745,0.647540,0.954819,0.000002,0.325021,0.263240,world/life,1.0
2,6,johnnie ray,cry,1950,pop,sweetheart send letter goodbye secret feel bet...,24,0.002770,0.002770,0.002770,...,0.002770,0.225422,0.456298,0.585288,0.840361,0.000000,0.351814,0.139112,music,1.0
3,10,pérez prado,patricia,1950,pop,kiss lips want stroll charm mambo chacha merin...,54,0.048249,0.001548,0.001548,...,0.225889,0.001548,0.686992,0.744404,0.083935,0.199393,0.775350,0.743736,romantic,1.0
4,12,giorgos papadopoulos,apopse eida oneiro,1950,pop,till darling till matter know till dream live ...,48,0.001350,0.001350,0.417772,...,0.068800,0.001350,0.291671,0.646489,0.975904,0.000246,0.597073,0.394375,romantic,1.0


## Remoção das colunas que não serão utilizadas

In [9]:
dataset.columns

Index(['Unnamed: 0', 'artist_name', 'track_name', 'release_date', 'genre',
       'lyrics', 'len', 'dating', 'violence', 'world/life', 'night/time',
       'shake the audience', 'family/gospel', 'romantic', 'communication',
       'obscene', 'music', 'movement/places', 'light/visual perceptions',
       'family/spiritual', 'like/girls', 'sadness', 'feelings', 'danceability',
       'loudness', 'acousticness', 'instrumentalness', 'valence', 'energy',
       'topic', 'age'],
      dtype='object')

In [15]:
dataset.drop(columns=["release_date","len", 'dating', 'violence', 'world/life', 'night/time',
       'shake the audience', 'family/gospel', 'romantic', 'communication',
       'obscene', 'music', 'movement/places', 'light/visual perceptions',
       'family/spiritual', 'like/girls', 'sadness', 'feelings', 'danceability',
       'loudness', 'acousticness', 'instrumentalness', 'valence', 'energy',
       'topic', 'age', 'artist_name'], inplace=True)

In [16]:
dataset.head()

,Unnamed: 0,track_name,genre,lyrics
0,0,mohabbat bhi jhoothi,pop,hold time feel break feel untrue convince spea...
1,4,i believe,pop,believe drop rain fall grow believe darkest ni...
2,6,cry,pop,sweetheart send letter goodbye secret feel bet...
3,10,patricia,pop,kiss lips want stroll charm mambo chacha merin...
4,12,apopse eida oneiro,pop,till darling till matter know till dream live ...
